In [99]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "04" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 4
repetitionName = "udrlBC4CVTest"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.1
print(len(savedResults))
print(len(savedResults)/9)

16
1.7777777777777777


In [100]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 1155)
(10, 1155)


0         1         2         3         4         5         6     \
0      None      None      None      None      None      None      None   
1    0.2125      0.25    0.2125      0.55     0.575    0.5375      0.65   
2      0.25    0.1875  0.177083  0.635417  0.572917  0.677083  0.645833   
3  0.305556      0.25  0.319444  0.541667  0.541667  0.486111  0.569444   
4  0.208333  0.229167  0.197917  0.677083   0.59375  0.635417  0.614583   
5  0.333333  0.291667  0.333333   0.65625  0.614583    0.6875    0.6875   
6  0.238636  0.238636      0.25  0.613636  0.545455  0.636364  0.602273   
7   0.28125  0.208333  0.229167     0.625   0.59375    0.5625  0.666667   
8      0.25     0.275      0.15     0.725      0.65       0.7       0.7   
9  0.229167  0.270833  0.166667  0.354167    0.4375  0.520833    0.5625   

       7         8         9         10        11        12        13    \
0      None      None      None      None      None      None      None   
1    0.5375     0.625     0.725     0.725      0.75     0.725    0.7375   
2  0.677083  0.604167    0.6875      0.75    0.6875  0.760417      0.75   
3  0.444444  0.611111  0.694444  0.694444  0.694444  0.736111  0.680556   
4    0.5625  0.614583  0.739583   0.71875      0.75   0.65625  0.677083   
5  0.604167  0.677083  0.760417  0.666667  0.677083  0.697917  0.708333   
6  0.579545  0.647727  0.693182     0.625  0.693182  0.670455  0.693182   
7     0.625  0.677083  0.635417     0.625  0.666667  0.708333    0.6875   
8       0.7       0.7       0.7       0.7       0.7      0.65       0.7   
9  0.416667  0.541667  0.708333    0.8125      0.75  0.708333    0.6875   

       14        15        16        17        18        19        20    \
0      None      None      None      None      None      None      None   
1    0.6875    0.6125     0.625    0.6375      0.25    0.2125    0.1625   
2    0.6875  0.635417  0.645833  0.635417  0.177083  0.208333  0.208333   
3  0.736111  0.555556  0.666667  0.541667  0.208333  0.319444  0.194444   
4    0.6875   0.65625   0.53125  0.645833  0.229167  0.166667  0.145833   
5  0.697917  0.583333  0.583333  0.677083   0.28125  0.208333  0.166667   
6  0.715909  0.613636  0.659091  0.636364  0.170455      0.25  0.227273   
7    0.6875  0.614583  0.604167  0.645833  0.239583      0.25      0.25   
8     0.675       0.7     0.625     0.675     0.325     0.325      0.35   
9  0.645833    0.5625  0.583333  0.416667  0.208333  0.229167    0.1875   

       21        22        23        24        25        26        27    \
0      None      None      None      None      None      None      None   
1       0.7    0.6125    0.6375     0.475      0.55    0.6875     0.525   
2  0.614583     0.625    0.6875  0.572917  0.604167  0.614583   0.40625   
3  0.708333  0.652778  0.694444  0.597222  0.527778       0.5       0.5   
4  0.614583  0.645833  0.635417    0.5625  0.416667  0.583333  0.458333   
5     0.625   0.59375  0.708333  0.614583  0.541667  0.583333  0.604167   
6  0.670455  0.590909  0.681818  0.579545  0.556818  0.556818  0.340909   
7   0.65625    0.6875   0.65625  0.614583  0.572917  0.541667  0.520833   
8       0.7       0.7       0.7     0.625     0.625     0.575       0.7   
9  0.479167  0.229167    0.4375  0.458333  0.479167  0.458333  0.291667   

       28        29        30        31        32        33        34    \
0      None      None      None      None      None      None      None   
1     0.325      0.45       0.5    0.5125     0.425    0.2125      0.25   
2  0.385417    0.4375  0.510417  0.520833  0.541667      0.25    0.1875   
3  0.458333  0.527778  0.472222  0.430556  0.472222  0.305556  0.277778   
4    0.4375    0.4375  0.572917  0.552083  0.552083  0.208333  0.197917   
5  0.510417  0.614583  0.645833  0.645833  0.604167  0.364583     0.375   
6  0.431818  0.340909  0.465909  0.431818  0.488636  0.238636  0.215909   
7  0.479167   0.46875  0.541667  0.583333   0.46875   0.28125  0.177083   
8     0.475     0.725     0.

In [101]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.83 : subject 9, feature:dataHRBC-welchDataBC-fftDataBC--fftData, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-gaussianDataBC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC-dataWCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC--fftData, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC--dataWCV-BC, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC-dataGCVBC-fftDataBC--fftData, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC--fftDataBC, kernelrbf, C:2.5


In [102]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.83 : subject 9, feature:dataGCV-BC-fftDataBC--fftData, kernelrbf, C:2.5
Accuracy 0.83 : subject 9, feature:dataGCV-BC--fftDataBC, kernelrbf, C:2.5

Accuracy 0.8 : subject 5, feature:dataHRBC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.8 : subject 5, feature:dataHRBC--fftDataBC, kernellinear, C:0.5

Accuracy 0.79 : subject 1, feature:dataGCV-BC-gaussianDataBC-fftDataBC--dataFFTCV-BC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 1, feature:dataGCV-BC-gaussianDataBC--fftDataBC, kernelsigmoid, C:2.5

Accuracy 0.78 : subject 2, feature:welchDataBC-dataFFTCV-BC--fftData, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 2, feature:dataGCV-BC-welchDataBC--dataWCV-BC, kernelsigmoid, C:2.5

Accuracy 0.78 : subject 8, feature:dataGCV-BC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 8, feature:dataGCV-BC--fftDataBC, kernellinear, C:0.5

Accuracy 0.77 : subject 4, feature:welchDataBC-fftDataBC--fftData, kernellinear, C:0.5
Accura

In [103]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.65540043 0.65912698 0.64083694 0.6463925  0.67203283
 0.62590516 0.63013167 0.67419913 0.5979798 ]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

Max average accuracy 0.7368897306397306
1077
9.0
dataGCV-BC-gaussianDataBC-welchDataBC--fftDataBC
linear
C: 0.5


In [104]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-fftData
1
-dataFFTCV-BC
2
-dataWCV-BC
3
-fftDataBC
4
-welchDataBC
5
-dataHRBC
6
-gaussianData
7
-dataGCVBC
8
-gaussianDataBC
9
-dataGCV-BC
10
dataFFTCV-BC--fftData
11
dataWCV-BC--fftData
12
fftDataBC--fftData
13
welchDataBC--fftData
14
dataHRBC--fftData
15
gaussianData--fftData
16
dataGCVBC--fftData
17
gaussianDataBC--fftData
18
dataGCV-BC--fftData
19
dataWCV-BC--dataFFTCV-BC
20
fftDataBC--dataFFTCV-BC
21
welchDataBC--dataFFTCV-BC
22
dataHRBC--dataFFTCV-BC
23
gaussianData--dataFFTCV-BC
24
dataGCVBC--dataFFTCV-BC
25
gaussianDataBC--dataFFTCV-BC
26
dataGCV-BC--dataFFTCV-BC
27
fftDataBC--dataWCV-BC
28
welchDataBC--dataWCV-BC
29
dataHRBC--dataWCV-BC
30
gaussianData--dataWCV-BC
31
dataGCVBC--dataWCV-BC
32
gaussianDataBC--dataWCV-BC
33
dataGCV-BC--dataWCV-BC
34
welchDataBC--fftDataBC
35
dataHRBC--fftDataBC
36
gaussianData--fftDataBC
37
dataGCVBC--fftDataBC
38
gaussianDataBC--fftDataBC
39
dataGCV-BC--fftDataBC
40
dataHRBC--welchDataBC
41
gaussianData--welchDataBC
42
dataGCVBC--welchDataBC
43

In [105]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

1077
9.0
Max total significant accuracy then averaged 0.7368897306397306 
dataGCV-BC-gaussianDataBC-welchDataBC--fftDataBC
linear
C: 0.5 


In [106]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
